# 評価例：プッシュ通知要約プロンプトの回帰

評価は**タスク指向**で反復的であり、LLM統合の状況を確認し改善するための最良の方法です。

以下の評価では、**プロンプト変更が回帰かどうかを検出する**タスクに焦点を当てます。

私たちのユースケースは以下の通りです：
1. プッシュ通知のリストを受け取り、それらを単一の要約文にまとめるLLM統合を持っている
2. プロンプト変更が動作を回帰させるかどうかを検出したい

## 評価の構造

評価には「Eval」と「Run」の2つの部分があります。「Eval」はテスト基準の設定と「Run」のデータ構造を保持します。Evalは、テスト基準によって評価される多くのRunを持つことができます。

In [1]:
import openai
from openai.types.chat import ChatCompletion
import pydantic
import os

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY", "your-api-key")

## ユースケース

以下の統合をテストしています。これは複数のプッシュ通知を受け取り、それらを単一の通知にまとめるプッシュ通知サマリー機能で、chat completions呼び出しを使用します。

In [ ]:
class PushNotifications(pydantic.BaseModel):
    notifications: str

print(PushNotifications.model_json_schema())

In [ ]:
DEVELOPER_PROMPT = """
You are a helpful assistant that summarizes push notifications.
You are given a list of push notifications and you need to collapse them into a single one.
Output only the final summary, nothing else.
"""

def summarize_push_notification(push_notifications: str) -> ChatCompletion:
    result = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "developer", "content": DEVELOPER_PROMPT},
            {"role": "user", "content": push_notifications},
        ],
    )
    return result

example_push_notifications_list = PushNotifications(notifications="""
- Alert: Unauthorized login attempt detected.
- New comment on your blog post: "Great insights!"
- Tonight's dinner recipe: Pasta Primavera.
""")
result = summarize_push_notification(example_push_notifications_list.notifications)
print(result.choices[0].message.content)

# Evalの設定

Evalは複数の*Run*間で共有される設定を保持し、2つのコンポーネントがあります：
1. データソース設定 `data_source_config` - 将来の*Run*が準拠するスキーマ（列）。
    - `data_source_config`はJSON Schemaを使用して、Evalで利用可能な変数を定義します。
2. テスト基準 `testing_criteria` - データソースの各*行*に対して統合が機能しているかどうかを判断する方法。

このユースケースでは、プッシュ通知の要約完了が良好かどうかをテストしたいので、これを念頭に置いてevalを設定します。

In [4]:
# We want our input data to be available in our variables, so we set the item_schema to
# PushNotifications.model_json_schema()
data_source_config = {
    "type": "custom",
    "item_schema": PushNotifications.model_json_schema(),
    # We're going to be uploading completions from the API, so we tell the Eval to expect this
    "include_sample_schema": True,
}

この`data_source_config`は、評価全体を通して利用可能な変数を定義します。

このアイテムスキーマ：
```json
{
  "properties": {
    "notifications": {
      "title": "Notifications",
      "type": "string"
    }
  },
  "required": ["notifications"],
  "title": "PushNotifications",
  "type": "object"
}
```
これは、評価で変数`{{item.notifications}}`が利用可能になることを意味します。

`"include_sample_schema": True`
これは、評価で変数`{{sample.output_text}}`が利用可能になることを意味します。

**次に、これらの変数を使用してテスト基準を設定します。**

In [5]:
GRADER_DEVELOPER_PROMPT = """
Label the following push notification summary as either correct or incorrect.
The push notification and the summary will be provided below.
A good push notificiation summary is concise and snappy.
If it is good, then label it as correct, if not, then incorrect.
"""
GRADER_TEMPLATE_PROMPT = """
Push notifications: {{item.notifications}}
Summary: {{sample.output_text}}
"""
push_notification_grader = {
    "name": "Push Notification Summary Grader",
    "type": "label_model",
    "model": "o3-mini",
    "input": [
        {
            "role": "developer",
            "content": GRADER_DEVELOPER_PROMPT,
        },
        {
            "role": "user",
            "content": GRADER_TEMPLATE_PROMPT,
        },
    ],
    "passing_labels": ["correct"],
    "labels": ["correct", "incorrect"],
}

`push_notification_grader`は、モデルグレーダー（llm-as-a-judge）であり、入力`{{item.notifications}}`と生成された要約`{{sample.output_text}}`を確認して、「correct」または「incorrect」としてラベル付けします。
その後、「passing_labels」を通じて、合格となる回答の構成要素を指示します。

注意：内部的には、これは構造化出力を使用しているため、ラベルは常に有効です。

**それでは評価を作成し、データの追加を開始しましょう！**

In [6]:
eval_create_result = openai.evals.create(
    name="Push Notification Summary Workflow",
    metadata={
        "description": "This eval checks if the push notification summary is correct.",
    },
    data_source_config=data_source_config,
    testing_criteria=[push_notification_grader],
)

eval_id = eval_create_result.id

# 実行の作成

テスト基準（test_criteria）を使用した評価のセットアップが完了したので、多数の実行を追加し始めることができます！
まずはプッシュ通知データから始めましょう。

In [7]:
push_notification_data = [
        """
- New message from Sarah: "Can you call me later?"
- Your package has been delivered!
- Flash sale: 20% off electronics for the next 2 hours!
""",
        """
- Weather alert: Thunderstorm expected in your area.
- Reminder: Doctor's appointment at 3 PM.
- John liked your photo on Instagram.
""",
        """
- Breaking News: Local elections results are in.
- Your daily workout summary is ready.
- Check out your weekly screen time report.
""",
        """
- Your ride is arriving in 2 minutes.
- Grocery order has been shipped.
- Don't miss the season finale of your favorite show tonight!
""",
        """
- Event reminder: Concert starts at 7 PM.
- Your favorite team just scored!
- Flashback: Memories from 3 years ago.
""",
        """
- Low battery alert: Charge your device.
- Your friend Mike is nearby.
- New episode of "The Tech Hour" podcast is live!
""",
        """
- System update available.
- Monthly billing statement is ready.
- Your next meeting starts in 15 minutes.
""",
        """
- Alert: Unauthorized login attempt detected.
- New comment on your blog post: "Great insights!"
- Tonight's dinner recipe: Pasta Primavera.
""",
        """
- Special offer: Free coffee with any breakfast order.
- Your flight has been delayed by 30 minutes.
- New movie release: "Adventures Beyond" now streaming.
""",
        """
- Traffic alert: Accident reported on Main Street.
- Package out for delivery: Expected by 5 PM.
- New friend suggestion: Connect with Emma.
"""]

最初の実行では、上記の completions 関数 `summarize_push_notification` からのデフォルトグレーダーを使用します。
データセットをループ処理し、completions 呼び出しを行い、それらを採点対象の実行として送信します。

In [ ]:
run_data = []
for push_notifications in push_notification_data:
    result = summarize_push_notification(push_notifications)
    run_data.append({
        "item": PushNotifications(notifications=push_notifications).model_dump(),
        "sample": result.model_dump()
    })

eval_run_result = openai.evals.runs.create(
    eval_id=eval_id,
    name="baseline-run",
    data_source={
        "type": "jsonl",
        "source": {
            "type": "file_content",
            "content": run_data,
        }
    },
)
print(eval_run_result)
# Check out the results in the UI
print(eval_run_result.report_url)

それでは回帰をシミュレートしてみましょう。これが元のプロンプトです。開発者がプロンプトを壊すシナリオをシミュレートしてみましょう。

```python
DEVELOPER_PROMPT = """
You are a helpful assistant that summarizes push notifications.
You are given a list of push notifications and you need to collapse them into a single one.
Output only the final summary, nothing else.
"""
```

In [9]:
DEVELOPER_PROMPT = """
You are a helpful assistant that summarizes push notifications.
You are given a list of push notifications and you need to collapse them into a single one.
You should make the summary longer than it needs to be and include more information than is necessary.
"""

def summarize_push_notification_bad(push_notifications: str) -> ChatCompletion:
    result = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "developer", "content": DEVELOPER_PROMPT},
            {"role": "user", "content": push_notifications},
        ],
    )
    return result

In [ ]:
run_data = []
for push_notifications in push_notification_data:
    result = summarize_push_notification_bad(push_notifications)
    run_data.append({
        "item": PushNotifications(notifications=push_notifications).model_dump(),
        "sample": result.model_dump()
    })

eval_run_result = openai.evals.runs.create(
    eval_id=eval_id,
    name="regression-run",
    data_source={
        "type": "jsonl",
        "source": {
            "type": "file_content",
            "content": run_data,
        }
    },
)
print(eval_run_result.report_url)

そのレポートを確認すると、ベースライン実行よりもはるかに低いスコアが表示されていることがわかります。

## おめでとうございます、ユーザーにバグが配信されることを防ぎました

注意点：
Evalsは現在`responses` APIをネイティブにサポートしていませんが、以下のコードを使用して`completions`形式に変換することができます。

In [ ]:
def summarize_push_notification_responses(push_notifications: str):
    result = openai.responses.create(
                model="gpt-4o",
                input=[
                    {"role": "developer", "content": DEVELOPER_PROMPT},
                    {"role": "user", "content": push_notifications},
                ],
            )
    return result
def transform_response_to_completion(response):
    completion = {
        "model": response.model,
        "choices": [{
        "index": 0,
        "message": {
            "role": "assistant",
            "content": response.output_text
        },
        "finish_reason": "stop",
    }]
    }
    return completion

run_data = []
for push_notifications in push_notification_data:
    response = summarize_push_notification_responses(push_notifications)
    completion = transform_response_to_completion(response)
    run_data.append({
        "item": PushNotifications(notifications=push_notifications).model_dump(),
        "sample": completion
    })

report_response = openai.evals.runs.create(
    eval_id=eval_id,
    name="responses-run",
    data_source={
        "type": "jsonl",
        "source": {
            "type": "file_content",
            "content": run_data,
        }
    },
)
print(report_response.report_url)